In [1]:
import torch
import numpy as np
import pandas as pd
# set to True to use the gpu (if there is one available)
use_gpu = True

# select device
if torch.cuda.is_available():
    device = torch.cuda.current_device()
else:
    device =  -1
print(device)

0


In [2]:
from datasets import load_from_disk
dataset = load_from_disk("data/belief_dataset/")

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at c:\Users\conni\Documents\csc583\483-583-project\data\belief_dataset\cache-4dffc2e6fb5da65a.arrow


In [5]:
data_dict = tokenized_datasets.train_test_split(test_size=0.95)
train_ds = data_dict['train']

all_test_ds = data_dict['test']
all_test_dict = all_test_ds.train_test_split(test_size=0.80)
eval_ds = all_test_dict['train']
test_ds = all_test_dict['test']


for entry in train_ds:
    print(entry['label'])

0
0
2
2
2
2
0
0
2
3
0
3
1
2
0
2
2
2


In [6]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="models/few-shot-bart-large-mnli")

import numpy as np
import evaluate
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


from transformers import TrainingArguments, Trainer
batch_size = 4
num_epochs = 2
training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  num_train_epochs=num_epochs,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size)

trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    compute_metrics=compute_metrics,
)

In [7]:
trainer.train()

c:\Users\conni\anaconda3\envs\tf-gpu\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 4.00 GiB total capacity; 3.41 GiB already allocated; 0 bytes free; 3.42 GiB reserved in total by PyTorch)